In [18]:
import random
import torch
torch.cuda.set_device(0)
import cv2
import numpy as np
from tools.run_infinity import *

# model_path='./weights/infinity_2b_reg.pth'
# model_path='/media/avlab/f09873b9-7c6a-4146-acdb-7db847b573201/VAR_ckpt/checkpoints/train/slim-ckpt-giter075K-ep1-iter34528-last.pth'
model_path='./weights/infinity_2b_reg.pth'
# model_path='./weights/infinity_125M.pth'   
# vae_path='./weights/infinity_vae_d16.pth'
vae_path='./weights/infinity_vae_d32_reg.pth'
text_encoder_ckpt = "./weights/models--google--flan-t5-xl/snapshots/7d6315df2c2fb742f0f5b556879d730926ca9001"

args=argparse.Namespace(
    pn='1M',
    model_path=model_path,
    cfg_insertion_layer=0,
    # vae_type=16,
    vae_type=32,
    vae_path=vae_path,
    add_lvl_embeding_only_first_block=1,
    use_bit_label=1,
    model_type='infinity_2b',
    # model_type='infinity_layer12',
    rope2d_each_sa_layer=1,
    rope2d_normalized_by_hw=2,
    use_scale_schedule_embedding=0,
    sampling_per_bits=1,
    text_encoder_ckpt=text_encoder_ckpt,
    text_channels=2048,
    apply_spatial_patchify=0,
    h_div_w_template=1.000,
    use_flex_attn=0,
    cache_dir='/media/avlab/f09873b9-7c6a-4146-acdb-7db847b573201/VAR_ckpt/checkpoints/train',
    checkpoint_type='torch',
    enable_model_cache=False, #modify 2 True when 26.9 GB model
    seed=0,
    bf16=1,
    save_file='new_tmp.jpg'
)

# Load model

In [19]:

# load text encoder
text_tokenizer, text_encoder = load_tokenizer(t5_path=args.text_encoder_ckpt)
# load vae
vae = load_visual_tokenizer(args)
# load infinity
infinity = load_transformer(vae, args)

[Loading tokenizer and text encoder]


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.23it/s]


load checkpoint from ./weights/infinity_2b_reg.pth
[Loading Infinity]
self.codebook_dim: 32, self.add_lvl_embeding_only_first_block: 1,             self.use_bit_label: 1, self.rope2d_each_sa_layer: 1, self.rope2d_normalized_by_hw: 2
self.num_blocks_in_a_chunk=4, depth=32, block_chunks=8

[constructor]  ==== customized_flash_attn=False (using_flash=0/32), fused_mlp=False (fused_mlp=0/32) ==== 
    [Infinity config ] embed_dim=2048, num_heads=16, depth=32, mlp_ratio=4, swiglu=False num_blocks_in_a_chunk=4
    [drop ratios] drop_rate=0.0, drop_path_rate=0.1 (tensor([0.0000, 0.0032, 0.0065, 0.0097, 0.0129, 0.0161, 0.0194, 0.0226, 0.0258,
        0.0290, 0.0323, 0.0355, 0.0387, 0.0419, 0.0452, 0.0484, 0.0516, 0.0548,
        0.0581, 0.0613, 0.0645, 0.0677, 0.0710, 0.0742, 0.0774, 0.0806, 0.0839,
        0.0871, 0.0903, 0.0935, 0.0968, 0.1000]))

[you selected Infinity with model_kwargs={'depth': 32, 'embed_dim': 2048, 'num_heads': 16, 'drop_path_rate': 0.1, 'mlp_ratio': 4, 'block_chunks': 8

## Try Other VAE

In [44]:
other_vae_pth = './weights/infinity_vae_d32_reg.pth'
# load vae
vae = load_visual_tokenizer(args)

## Gen image iteractively

In [ ]:
prompt = """
The image is taken from a slightly elevated angle, providing a clear view of the theater marquee. The marquee displays the text \"ANNO DOMINI\" in large, bold letters at the top, followed by \"SANER\" in smaller letters below. Further down, it reads \"MAR 6 THRU APR 18\" indicating the dates of the event. The background consists of a gray brick wall with some greenery peeking through at the top. The foreground features several hanging street lamps, adding to the urban setting. The overall composition suggests a daytime scene with natural lighting.
"""
cfg = 3
tau = 0.5
h_div_w = 1/1 # aspect ratio, height:width
seed = random.randint(0, 10000)
enable_positive_prompt=0

h_div_w_template_ = h_div_w_templates[np.argmin(np.abs(h_div_w_templates-h_div_w))]
scale_schedule = dynamic_resolution_h_w[h_div_w_template_][args.pn]['scales']
scale_schedule = [(1, h, w) for (_, h, w) in scale_schedule]
generated_image = gen_one_img(
    infinity,
    vae,
    text_tokenizer,
    text_encoder,
    prompt,
    g_seed=seed,
    gt_leak=0,
    gt_ls_Bl=None,
    cfg_list=cfg,
    tau_list=tau,
    scale_schedule=scale_schedule,
    cfg_insertion_layer=[args.cfg_insertion_layer],
    vae_type=args.vae_type,
    sampling_per_bits=args.sampling_per_bits,
    enable_positive_prompt=enable_positive_prompt,
)
args.save_file = '2b_1M.jpg'
os.makedirs(osp.dirname(osp.abspath(args.save_file)), exist_ok=True)
cv2.imwrite(args.save_file, generated_image.cpu().numpy())
print(f'Save to {osp.abspath(args.save_file)}')

prompt=
The image is taken from a slightly elevated angle, providing a clear view of the theater marquee. The marquee displays the text "ANNO DOMINI" in large, bold letters at the top, followed by "SANER" in smaller letters below. Further down, it reads "MAR 6 THRU APR 18" indicating the dates of the event. The background consists of a gray brick wall with some greenery peeking through at the top. The foreground features several hanging street lamps, adding to the urban setting. The overall composition suggests a daytime scene with natural lighting.

cfg: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], tau: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
cost: 1.96470046043396, infinity cost=1.9080650806427002
Save to /home/avlab/SceneTxtVAR/2b_1M.jpg


: 

In [13]:
prompt = """
Ultra‑realistic 35 mm photograph of an outdoor public swimming pool at midday:  In the foreground, fixed to a stainless‑steel safety fence, a slightly weather‑worn white sign with bold, uneven red lettering that reads “NO WEARING SHOSES”, edges peeling and casting a delicate shadow on the slick tiles. High dynamic range, authentic colors, subtle grain, cinematic realism.
"""
# prompt = """
# The image is a digitally rendered image viewed from a slightly elevated angle, showcasing a modern bedroom setup. The bed \"aeruginose\" is subtly visible at the bottom of the image in a white, minimalist font. The scene features a low-slung bed with gray upholstery and a pink throw blanket at its center, flanked by a pair of sleek bedside tables. The wall behind the bed is adorned with colorful abstract art pieces. To the right, a gray storage unit with glassware and decorative objects sits under a hanging plant. Large windows with blinds allow natural light to pour in, illuminating the wooden floor with a decorative pattern at its center. The ceiling combines wood beams with white panels, while a section white accent wall adds contrast to the room's bright ambiance.
# """
cfg = 3
tau = 0.5
h_div_w = 0.75 # aspect ratio, height:width
seed = random.randint(0, 10000)
enable_positive_prompt=0

h_div_w_template_ = h_div_w_templates[np.argmin(np.abs(h_div_w_templates-h_div_w))]
scale_schedule = dynamic_resolution_h_w[h_div_w_template_][args.pn]['scales']
scale_schedule = [(1, h, w) for (_, h, w) in scale_schedule]
generated_image = gen_one_img(
    infinity,
    vae,
    text_tokenizer,
    text_encoder,
    prompt,
    g_seed=seed,
    gt_leak=0,
    gt_ls_Bl=None,
    cfg_list=cfg,
    tau_list=tau,
    scale_schedule=scale_schedule,
    cfg_insertion_layer=[args.cfg_insertion_layer],
    vae_type=args.vae_type,
    sampling_per_bits=args.sampling_per_bits,
    enable_positive_prompt=enable_positive_prompt,
)
args.save_file = 'train_2.jpg'
os.makedirs(osp.dirname(osp.abspath(args.save_file)), exist_ok=True)
cv2.imwrite(args.save_file, generated_image.cpu().numpy())
print(f'Save to {osp.abspath(args.save_file)}')

prompt=
Ultra‑realistic 35 mm photograph of an outdoor public swimming pool at midday:  In the foreground, fixed to a stainless‑steel safety fence, a slightly weather‑worn white sign with bold, uneven red lettering that reads “NO WEARING SHOSES”, edges peeling and casting a delicate shadow on the slick tiles. High dynamic range, authentic colors, subtle grain, cinematic realism.

cfg: [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], tau: [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
cost: 2.045748233795166, infinity cost=1.983825922012329
Save to /home/avlab/SceneTxtVAR/train_2.jpg
